# Create a simple ISA descriptor

This example details:
- How to create metadata for a single study ISA descriptor
- How to write the ISA in-memory Model content to file in ISA-Tab format.
- How to write the ISA in-memory Model to file in ISA-JSON format.
- How to load an ISA object representation from an ISA-JSON document on disk.

## ISA Study metadata

In [1]:
from isatools.model import *
from isatools.isatab import *
from isatools.isajson import *
import io
# from isatools.isajson import ISAJSONEncoder
# from isatools.isajson import load
# from isatools.isatab import dumps

from ipywidgets import RadioButtons, VBox, HBox, Layout, Label, Text, Textarea, DatePicker

import json
import datetime
import os

identifier = Text(
        value="1",
        placeholder='e.g. 1',
        description='Identifier:',
        disabled=False
    )
title = Text(
        value='My Simple ISA Study',
        placeholder='e.g. My Study title',
        description='Title:',
        disabled=False
    )
description = Textarea(
    value="We could alternatively use the class constructor's parameters to set some default " \
          "values at the time of creation, however we want to demonstrate how to use the " \
          "object's instance variables to set values.",
    placeholder='Type something',
    description='Description:',
    disabled=False
)
sub_date = DatePicker(description="Submission date:", value=datetime.datetime.today())
release_date = DatePicker(description="Public release date:", value=datetime.datetime.today())
VBox([identifier,
      title,
      description,
      sub_date,
      release_date])

### 1. Declaring terminology resources 

In [2]:
investigation = Investigation(identifier=1)

obi = OntologySource(name='OBI', description="Ontology for Biomedical Investigation")
ncbitaxon = OntologySource(name='NCBITaxon', description="NCBI Taxonomy")
# 2. Adding them to the Investigation object
investigation.ontology_source_references.append(obi)
investigation.ontology_source_references.append(ncbitaxon)

# print(investigation)


### 2. Creating an example of ISA Study descriptor:

In [3]:
study_design_annotation = OntologyAnnotation(term_source=obi)
study_design_annotation.term = "intervention design"
study_design_annotation.term_accession = "http://purl.obolibrary.org/obo/OBI_0000115"

### 3. Populating some of the ISA Study attribute from the form:

In [4]:
study = Study(filename="s_study.txt")
study.identifier = identifier.value
study.title = title.value
study.description = description.value
study.submission_date = str(sub_date.value)
study.public_release_date = str(release_date.value)
study.design_descriptors.append(study_design_annotation)

### 4. Adding an ISA Study to an ISA Investigation object

In [5]:
investigation.studies = [study]
print(study)

# study.comments.append(Comment(name="Study Start Date",value="Sun"))
# adding a dummy Comment[] to ISA.protocol object
# study.protocols[0].comments.append(Comment(name="Study Start Date",value="Uranus"))
# checking that the ISA Protocool object has been modified
# print(study.protocols[0])

Study(
    identifier=1
    filename=s_study.txt
    title=My Simple ISA Study
    description=We could alternatively use the class constructor's parameters to set some default values at the time of creation, however we want to demonstrate how to use the object's instance variables to set values.
    submission_date=2019-08-22 15:57:42.911377
    public_release_date=2019-08-22 15:57:42.915263
    contacts=0 Person objects
    design_descriptors=1 OntologyAnnotation objects
    publications=0 Publication objects
    factors=0 StudyFactor objects
    protocols=0 Protocol objects
    assays=0 Assay objects
    sources=0 Source objects
    samples=0 Sample objects
    process_sequence=0 Process objects
    other_material=0 Material objects
    characteristic_categories=0 OntologyAnnotation objects
    comments=0 Comment objects
    units=0 Unit objects
)


### 5. Creation of an ISA Study Factor object

In [6]:
f=StudyFactor(name="treatment['modality']", factor_type=OntologyAnnotation(term="treatment['modality']"))
# checking that the ISA Factor object has been modified
study.factors.append(f)

#testing serialization to ISA-TAB of Comments attached to ISA objects.
# f.comments.append(Comment(name="Study Start Date",value="Moon"))
# print(f.comments[0].name, "|", f.comments[0].value)

### 6. Creation of an ISA Person object and adding it to the list of ISA Study Contacts:

In [7]:
contact = Person(first_name="Alice", last_name="Robertson", affiliation="University of Life", roles=[OntologyAnnotation(term='submitter')])
study.contacts.append(contact)

### 7. Creation of an ISA Publication object and adding it to the list of ISA Study Publications.

In [8]:
publication = Publication(title="Experiments with Humans", author_list="A. Robertson, B. Robertson")
publication.pubmed_id = "12345678"
publication.status = OntologyAnnotation(term="published")
study.publications.append(publication)

print(study)

Study(
    identifier=1
    filename=s_study.txt
    title=My Simple ISA Study
    description=We could alternatively use the class constructor's parameters to set some default values at the time of creation, however we want to demonstrate how to use the object's instance variables to set values.
    submission_date=2019-08-22 15:57:42.911377
    public_release_date=2019-08-22 15:57:42.915263
    contacts=1 Person objects
    design_descriptors=1 OntologyAnnotation objects
    publications=1 Publication objects
    factors=1 StudyFactor objects
    protocols=0 Protocol objects
    assays=0 Assay objects
    sources=0 Source objects
    samples=0 Sample objects
    process_sequence=0 Process objects
    other_material=0 Material objects
    characteristic_categories=0 OntologyAnnotation objects
    comments=0 Comment objects
    units=0 Unit objects
)


### 8. Creating a parent ISA Source:

In [9]:
source = Source(name='source_material')

In [10]:
#Specifying the taxonomic group of the ISA Source 
characteristic_organism = Characteristic( category=OntologyAnnotation(id_='1',
                                                                      term="Organism"),
                                                                      value=OntologyAnnotation(
                                                                             term="Homo Sapiens",
                                                                             term_source=ncbitaxon,
                                                                             term_accession="http://purl.bioontology.org/ontology/NCBITAXON/9606"))
# Adding the description to the ISA Source Material:
source.characteristics.append(characteristic_organism)

study.sources.append(source)

In [11]:
# Creating a template for ISA Sample Material deriving from the ISA Source Material we create before:
prototype_sample = Sample(name='sample_material', derives_from=[source])

# To annotate this sample, we need terms from a new ontology, here UBERON, let's declare it.
uberon = OntologySource(name='UBERON', description="UBER Anatomy Ontology")
# Important!: be sure to add all ontology resources, failing to do so will raise error when invoking a non-referenced resources
investigation.ontology_source_references.append(uberon)

# Creating an ```ISA characteristic``` attribute to annotation the ```ISA Sample``` to indicate the nature of the specimen
characteristic_organ = Characteristic(category=OntologyAnnotation(id_='2',
                                                                  term="OrganismPart"),
                                      value=OntologyAnnotation(term="liver",
                                                              term_source=uberon,
                                                              term_accession="http://purl.bioontology.org/ontology/UBERON/10031"))
# Creating another ```ISA Characteristic``` for testing:
char_test = Characteristic(category=OntologyAnnotation(id_="001",
                                                       term="test_category"),
                           value=OntologyAnnotation(term="test_value"))

# Associating the instances of 
prototype_sample.characteristics.append(characteristic_organ)
prototype_sample.characteristics.append(char_test)

study.samples = batch_create_materials(prototype_sample, n=3)

In [12]:
sample_collection_protocol = Protocol(name="sample collection",
                                      protocol_type=OntologyAnnotation(term="sample collection"))

study.protocols.append(sample_collection_protocol)

sample_collection_process = Process(executes_protocol=sample_collection_protocol)

for src in study.sources:
    sample_collection_process.inputs.append(src)
for sam in study.samples:
    sample_collection_process.outputs.append(sam)

study.process_sequence.append(sample_collection_process)

### Important: 
when creating the ISA descriptor objects manually, all the ISA Material Characteristics Category (the main type of annotations) need to be listed and attached to the ISA Study object, as shown below:

In [13]:
study.characteristic_categories.append(characteristic_organ.category)
study.characteristic_categories.append(characteristic_organism.category)
study.characteristic_categories.append(char_test.category)

### Creating ISA assays:

In [14]:
assay = Assay(filename="a_assay.txt")
extraction_protocol = Protocol(name='extraction', protocol_type=OntologyAnnotation(term="material extraction"))
study.protocols.append(extraction_protocol)
sequencing_protocol = Protocol(name='sequencing', protocol_type=OntologyAnnotation(term="material sequencing"))
study.protocols.append(sequencing_protocol)
    
for i, sample in enumerate(study.samples):
    
    # create an extraction process that executes the extraction protocol
    extraction_process = Process(executes_protocol=extraction_protocol)
    
    # extraction process takes as input a sample, and produces an extract material as output
    extraction_process.inputs.append(sample)
    material = Material(name="extract-{}".format(i))
    material.type = "Extract Name"
    extraction_process.outputs.append(material)

    # create a sequencing process that executes the sequencing protocol
    sequencing_process = Process(executes_protocol=sequencing_protocol)
    sequencing_process.name = "assay-name-{}".format(i)
    sequencing_process.inputs.append(extraction_process.outputs[0])

    # Sequencing process usually has an output data file
    datafile = DataFile(filename="sequenced-data-{}".format(i), label="Raw Data File", generated_from=[sample])
    sequencing_process.outputs.append(datafile)

    # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
    # these links for you. It is found in the isatools.model package
    plink(extraction_process, sequencing_process)

    # make sure the extract, data file, and the processes are attached to the assay
    assay.samples.append(sample)
    assay.data_files.append(datafile)
    assay.other_material.append(material)
    assay.process_sequence.append(extraction_process)
    assay.process_sequence.append(sequencing_process)

assay.measurement_type = OntologyAnnotation(term="gene sequencing")
assay.technology_type = OntologyAnnotation(term="nucleotide sequencing") 
    
study.assays.append(assay)

### Serialization as ISA-Tab

In [15]:
print(dumps(investigation))

Study Design Type                                               intervention design
Study Design Type Term Accession Number  http://purl.obolibrary.org/obo/OBI_0000115
Study Design Type Term Source REF                                               OBI
Study Factor Name                        treatment['modality']
Study Factor Type                        treatment['modality']
Study Factor Type Term Accession Number                       
Study Factor Type Term Source REF                             
/var/folders/5n/rl6lqnks4rqb59pbtpvvntqw0000gr/T/tmp8_xt8368/i_investigation.txt
ONTOLOGY SOURCE REFERENCE
Term Source Name	OBI	NCBITaxon	UBERON
Term Source File			
Term Source Version			
Term Source Description	Ontology for Biomedical Investigation	NCBI Taxonomy	UBER Anatomy Ontology
INVESTIGATION
Investigation Identifier	1
Investigation Title	
Investigation Description	
Investigation Submission Date	
Investigation Public Release Date	
INVESTIGATION PUBLICATIONS
Investigation PubMed ID
Inve

### Serialization as ISA-JSON

In [16]:
isa_j = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
print(isa_j)

{
    "comments": [],
    "description": "",
    "identifier": 1,
    "ontologySourceReferences": [
        {
            "description": "Ontology for Biomedical Investigation",
            "file": "",
            "name": "OBI",
            "version": ""
        },
        {
            "description": "NCBI Taxonomy",
            "file": "",
            "name": "NCBITaxon",
            "version": ""
        },
        {
            "description": "UBER Anatomy Ontology",
            "file": "",
            "name": "UBERON",
            "version": ""
        }
    ],
    "people": [],
    "publicReleaseDate": "",
    "publications": [],
    "studies": [
        {
            "assays": [
                {
                    "characteristicCategories": [],
                    "comments": [],
                    "dataFiles": [
                        {
                            "@id": "#data/rawdatafile-4582896696",
                            "comments": [],
                           

### Writing to file and closing the filehandle.

In [17]:
with open( 'isa.json', 'w') as out_fp:
    json.dump(isa_j, out_fp)
out_fp.close

<function TextIOWrapper.close>

### Roundtripping: Reading back into ISA object after having dumped the test Study as ISA JSON


In [18]:
with open( "isa.json", 'r') as input:
    try:
        jinput = json.loads(input.read())

        #IMPORTANT: do not confuse JSON loads with ISA load function with reads a filehandle to an ISA JSON document into ISA objects.    
        inv=load(io.StringIO(jinput))
        print(inv)
        print(inv.studies[0])
        print(inv.studies[0].characteristic_categories[1])
        print(inv.studies[0].sources[0].characteristics[0].category.term, ": ", inv.studies[0].sources[0].characteristics[0].value.term)
    except IOError as ioe:
            print(ioe)

Investigation(
    identifier=1
    filename=
    title=
    submission_date=
    public_release_date=
    ontology_source_references=3 OntologySource objects
    publications=0 Publication objects
    contacts=0 Person objects
    studies=1 Study objects
    comments=0 Comment objects
)
Study(
    identifier=1
    filename=s_study.txt
    title=My Simple ISA Study
    description=We could alternatively use the class constructor's parameters to set some default values at the time of creation, however we want to demonstrate how to use the object's instance variables to set values.
    submission_date=2019-08-22 15:57:42.911377
    public_release_date=2019-08-22 15:57:42.915263
    contacts=1 Person objects
    design_descriptors=1 OntologyAnnotation objects
    publications=1 Publication objects
    factors=1 StudyFactor objects
    protocols=3 Protocol objects
    assays=1 Assay objects
    sources=1 Source objects
    samples=3 Sample objects
    process_sequence=1 Process objects
   